In [23]:
import numpy as np
import pandas as pd

SyntaxError: invalid syntax (2819257074.py, line 3)

In [10]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

n

300

In [11]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

,n1,n2,e,w
0,1,2,0,1
1,1,3,0,2
2,1,4,0,6
3,1,5,0,1
4,1,6,0,3
...,...,...,...,...
44845,297,299,0,1
44846,297,300,0,1
44847,298,299,0,4
44848,298,300,0,2


In [12]:
# create nodes data frame
nodes_from = df.loc[df["e"]==1][["n1","w"]].groupby(['n1']).sum()
nodes_from
nodes_to = df.loc[df["e"]==1][["n2","w"]].groupby(['n2']).sum()
nodes_to

nodes = nodes_from.join(nodes_to, lsuffix='_from', rsuffix='_to', how = 'outer')
nodes['node_impact'] = nodes.w_from.fillna(0) + nodes.w_to.fillna(0)
nodes = nodes.drop(columns=['w_from', 'w_to'])
nodes['current_degree'] = 0
nodes['splex'] = nodes.index
nodes

,node_impact,current_degree,splex
1,431.0,0,1
2,115.0,0,2
3,46.0,0,3
4,113.0,0,4
5,111.0,0,5
...,...,...,...
296,6.0,0,296
297,165.0,0,297
298,50.0,0,298
299,385.0,0,299


In [13]:
edges = df
edges['w'] = edges['w'] * (1-(edges['e']*2))
edges['e'] = 0
edges.loc[edges['w'] < 0]

,n1,n2,e,w
8,1,10,0,-5
12,1,14,0,-1
13,1,15,0,-3
14,1,16,0,-1
16,1,18,0,-2
...,...,...,...,...
44827,293,299,0,-1
44828,293,300,0,-3
44831,294,297,0,-1
44833,294,299,0,-2


In [28]:
def is_splex(nodes, plex_number, s) -> bool | pd.DataFrame:
    splex = nodes.loc[nodes["splex"] == plex_number]
    min_degree = len(splex.index) - s
    problem_nodes = splex.loc[splex["current_degree"] < min_degree]
    
    if len(problem_nodes.index) == 0:
        return True
    else:
        return problem_nodes

is_splex(nodes, 2, 1)

True